<b><h2 style="text-align:center;">COMM493 - Coding AI for Business</h2></b>
<h5 style="text-align:center;">Assignment 2 - Text Classification Via Natural Language Processing</h5>
<h5 style="text-align:center;">Maxwell Brookes - 20244724</h5>
<h5 style="text-align:center;">March 1st, 2025</h5>

### Introduction
**Motivation:** TODO

**Data:** <a href="https://www.kaggle.com/datasets/abhishek14398/automatic-ticket-classification-dataset/">Automatic Ticket Classification Dataset</a> dataset contains ...
<a href="https://www.kaggle.com/code/abhishek14398/automatic-ticket-classification-case-study-nlp">implemenmtation</a>


**Goal:** Map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

### 0: Set Up Environment

In [1]:
from datetime import datetime
import pandas as pd
import json
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker import image_uris

# Initialize SageMaker
sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = 'complaints-classification-optimized'

# Precompile regex patterns
REDACTION_PATTERNS = re.compile(
    r'\b(?:X+X|X{2,}(?:/X{2,})+|\d+[-/]?X+|X+[-/]?\d+|X{4,})\b', 
    flags=re.IGNORECASE
)
CLEANING_PATTERNS = [
    (re.compile(r'(\\[nt])+'), ' '),
    (re.compile(r'\$ ?(\d+)'), r'\1 dollars'),
    (re.compile(r'\b(\d+)(?:st|nd|rd|th)\b'), r'\1'),
    (re.compile(r'[^\w\s]'), '')
]

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/10/25 00:35:14] INFO     Found credentials from IAM Role:                                   ]8;id=419900;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=870976;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=976824;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=645892;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=404714;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=25746;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

<b><h2 style="text-align:center;">DATA PREPROCESSING</h2></b>

### 1: Load dataset

In [2]:
with open('complaints.json', 'r') as f:
    data = json.load(f)
df = pd.json_normalize(data)
print('Data Shape:', df.shape)

Data Shape: (78313, 22)


### 2: Clean columns

In [3]:
# rename columns
df.rename(columns={
    '_source.product': 'category',
    '_source.complaint_what_happened': 'text'
}, inplace=True)

# drop columns
columns_to_keep = ['category', 'text']
all_columns = df.columns.tolist()
columns_to_drop = [col for col in all_columns if col not in columns_to_keep]
df.drop(columns_to_drop, axis=1, inplace=True)

df.columns

Index(['category', 'text'], dtype='object')

### 3: Clean rows and text

In [4]:
# null handling
df = df[
    df['text'].str.strip().astype(bool) &
    df['category'].notna() &
    df['category'].str.strip().astype(bool)
].copy()


# text cleaning
def clean_text_column(text_series):
    cleaned = text_series.str.replace(REDACTION_PATTERNS, '[REDACTED]')
    for pattern, replacement in CLEANING_PATTERNS:
        cleaned = cleaned.str.replace(pattern, replacement)
    return cleaned.str.lower().str.strip()


df['text'] = clean_text_column(df['text'])


# Stratified sampling with quantile-based balancing
category_counts = df['category'].value_counts()
min_samples = int(max(100, category_counts.quantile(0.85)))  # Ensure minimum 100 samples
balanced_dfs = []
for category, group in df.groupby('category'):
    if len(group) < min_samples:
        group = resample(group, replace=True, n_samples=min_samples, random_state=123)
    balanced_dfs.append(group)

df = pd.concat(balanced_dfs, ignore_index=True)
print(f"Balanced Data Shape: {df.shape}")

# Preview
df.head()

Balanced Data Shape: (51523, 2)


,category,text
0,Bank account or service,being charged erroneous bank fees on my checki...
1,Bank account or service,chase bank whom i have banked with foe redacte...
2,Bank account or service,received a chase quick pay for move in payment...
3,Bank account or service,chase bank has disabled my ability to make mob...
4,Bank account or service,jp morgan chase bank reordered my transactions...


### 4: Split and upload

In [5]:
train_data, validation_data = train_test_split(
    df, 
    test_size=0.2, 
    random_state=123,
    stratify=df['category']
)

def format_blazingtext(df):
    return '\n'.join(
        f"__label__{cat.replace(' ', '_')} {txt}" 
        for cat, txt in zip(df['category'], df['text'])
    )

# Batch write formatted data
for name, data in [('train', train_data), ('validation', validation_data)]:
    with open(f'{name}.txt', 'w') as f:
        f.write(format_blazingtext(data))

<b><h2 style="text-align:center;">MODEL TRAINING</h2></b>

### 5: Upload Data to S3

In [6]:
version = datetime.now().strftime("%Y%m%d-%H%M")
s3_prefix = f"{prefix}/{version}"

sagemaker_session.upload_data('train.txt', bucket=bucket, key_prefix=f'{s3_prefix}/train')
sagemaker_session.upload_data('validation.txt', bucket=bucket, key_prefix=f'{s3_prefix}/validation')

's3://sagemaker-us-east-1-922202922528/complaints-classification-optimized/20250310-0035/validation/validation.txt'

### 6: Train the BlazingText Model

In [7]:
region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    framework='blazingtext',
    region=region_name
)

# Enhanced hyperparameters
hyperparams = {
    "mode": "supervised",
    "epochs": 50,
    "learning_rate": 0.1,
    "min_count": 2,
    "vector_dim": 300,
    "word_ngrams": 3,
    "bucket": 200000,
    "early_stopping": True,
    "patience": 5,
    "threads": 8  # Utilize more CPU cores
}

# Configure estimator with optimized instance
bt_estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',  # Better compute ratio
    output_path=f's3://{bucket}/{s3_prefix}/output',
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparams
)

# Start training with versioned data
bt_estimator.fit({'train': f's3://{bucket}/{s3_prefix}/train/train.txt',
                  'validation': f's3://{bucket}/{s3_prefix}/validation/validation.txt'})

[03/10/25 00:35:23] INFO     Same images used for training and inference. Defaulting to image     ]8;id=933495;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=750180;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#393\393]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=84871;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=43832;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=359417;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=728137;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=293212;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=239773;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: blazingtext-2025-03-10-00-35-23-939   ]8;id=404414;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=410890;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-10 00:35:25 Starting - Starting the training job...
..25-03-10 00:35:40 Starting - Preparing the instances for training.
..25-03-10 00:36:25 Downloading - Downloading the training image.
2025-03-10 00:36:35 Training - Training image download completed. Training in progress.Arguments: train
[03/10/2025 00:36:49 WARNING 140664822634304] Loggers have already been setup.
[03/10/2025 00:36:49 WARNING 140664822634304] Loggers have already been setup.
/opt/amazon/python3.8/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
[03/10/2025 00:36:49 INFO 140664822634304] nvidia-smi took: 0.025221586227416992 secs to identify 0 gpus
[03/10/2025 00:36:49 INFO 140664822634304] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[03/10/2025 00:36:49 INFO 140664822634304] Processing /opt/ml

### 7: Deploy the Model for Inference

In [8]:
bt_predictor = bt_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name=f'complaint-classifier-{version}'
)

# Configure auto-scaling
client = boto3.client('application-autoscaling')
client.register_scalable_target(
    ServiceNamespace='sagemaker',
    ResourceId=f'endpoint/complaint-classifier-{version}/variant/AllTraffic',
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=3
)

[03/10/25 00:41:14] INFO     Creating model with name: blazingtext-2025-03-10-00-41-14-184          ]8;id=362292;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=463827;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name complaint-classifier-20250310-0035  ]8;id=845686;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=378785;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

[03/10/25 00:41:15] INFO     Creating endpoint with name complaint-classifier-20250310-0035         ]8;id=207665;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=783996;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-----!

{'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:922202922528:scalable-target/056m96d00916b9334d5f9c0e7765c88ceb95',
 'ResponseMetadata': {'RequestId': 'ca90d9c3-fc14-4f2a-a287-d6916c7a1017',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ca90d9c3-fc14-4f2a-a287-d6916c7a1017',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Mon, 10 Mar 2025 00:44:16 GMT'},
  'RetryAttempts': 0}}

### 8: Query Model

In [13]:
# Create test complaint
sample_complaint = "I have issues with my credit card payment being processed incorrectly."

# Get predictions for ALL categories
prediction = bt_predictor.predict({
    "instances": [sample_complaint],
    "configuration": {"k": 5}
})

# Process results
print("=== Complaint Text ===")
print(sample_complaint)
print(f"\n=== Predicted Categories (Top 5) ===")

labels = prediction[0]['label']
probs = prediction[0]['prob']

for rank, (label, prob) in enumerate(zip(labels, probs), 1):
    clean_label = label.replace('__label__', '').replace('_', ' ')
    print(f"Rank {rank}: {clean_label.ljust(30)} {prob*100:.2f}%")

=== Complaint Text ===
I have issues with my credit card payment being processed incorrectly.

=== Predicted Categories (Top 5) ===
Rank 1: Credit card or prepaid card    96.96%
Rank 2: Credit reporting, credit repair services, or other personal consumer reports 2.52%
Rank 3: Credit card                    0.42%
Rank 4: Debt collection                0.10%
Rank 5: Money transfer, virtual currency, or money service 0.00%


### 9: Clean Up

In [14]:
# Delete endpoint to avoid ongoing charges
bt_predictor.delete_endpoint()

[03/10/25 00:47:08] INFO     Deleting endpoint configuration with name:                             ]8;id=589943;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=101211;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\
                             complaint-classifier-20250310-0035                                                    

                    INFO     Deleting endpoint with name: complaint-classifier-20250310-0035        ]8;id=42379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=500535;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\